In [7]:
import tkinter as tk
import numpy as np
import time
from image_conversor import convert_image
import cv2

# Função para formatar a matriz como uma string de matriz
def format_matrix(matrix):
    return '\n'.join([' '.join(row) for row in matrix])

# Captura da câmera
cap = cv2.VideoCapture(0)

length = 5

# Verificar se a câmera foi aberta corretamente
if not cap.isOpened():
    print("Erro ao abrir a câmera.")
    cap.release()
    exit()

# Criar a janela do tkinter
root = tk.Tk()
root.title("Vídeo ASCII")
root.configure(bg="black")  # Definir o background para preto

# Criar um rótulo para exibir as matrizes na janela com fonte menor
label = tk.Label(root, text="", font=("Courier New", 10), bg="black", fg="#777777")
label.pack()

# Função para capturar cada frame da câmera e atualizar o rótulo com a matriz convertida em ASCII
def update_frame():
    while True:
        ret, frame = cap.read()

        if not ret:  # Verifica se a captura falhou ou o vídeo acabou
            break
        frame = convert_image(frame, length)
        formatted_matrix = format_matrix(frame)

        try:
            if root.winfo_exists():  # Verifica se a janela ainda está aberta
                label.config(text=formatted_matrix)
                root.update()
        except tk.TclError:
            break

# Chamar a função para exibir o vídeo da câmera em ASCII
update_frame()

cap.release()  # Liberar a captura da câmera após fechar a janela

root.mainloop()


In [6]:
import tkinter as tk
from PIL import Image, ImageDraw, ImageFont
import cv2
from image_conversor import convert_image

# Variável global para armazenar o frame capturado da câmera
frame = None

# Função para formatar a matriz como uma string de matriz
def format_matrix(matrix):
    return '\n'.join([' '.join(row) for row in matrix])

# Captura da câmera
cap = cv2.VideoCapture(0)

# Definir o comprimento da fonte
length = 5

# Função para capturar o próximo frame da câmera e atualizar a variável global frame
def capture_frame():
    global frame
    ret, frame = cap.read()
    root.after(100, capture_frame)  # Capturar o próximo frame a cada 100 milissegundos

# Função para atualizar o conteúdo da label com a matriz atualizada
def update_label():
    if frame is not None:
        converted_frame = convert_image(frame, length)
        formatted_matrix = format_matrix(converted_frame)
        label.config(text=formatted_matrix)
    root.after(100, update_label)  # Atualizar a label a cada 100 milissegundos

# Criar a janela do tkinter
root = tk.Tk()
root.title("Vídeo ASCII")
root.configure(bg="black")  # Definir o background para preto

# Criar um rótulo para exibir o conteúdo
label = tk.Label(root, text="", font=("Courier New", 10), bg="black", fg="#777777")
label.pack()

# Iniciar a captura de frames e a atualização da label
capture_frame()
update_label()

# Função para extrair o conteúdo da label como uma imagem e salvar em um arquivo PNG
def extract_image():
    width = label.winfo_width()
    height = label.winfo_height()
    image = Image.new("RGB", (width, height), color="black")
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()  # Utilizar a fonte padrão do PIL
    formatted_text = label.cget("text")  # Obter o texto atual da label
    draw.text((0, 0), formatted_text, fill="white", font=font)  # Usar o texto formatado
    image.save("output.png")

# Chamar a função para extrair a imagem automaticamente
root.after(100, extract_image)  # Esperar 100 milissegundos antes de chamar a função
cap.release()
root.mainloop()
